# 02 - Orbit Propagation (SGP4)

This notebook computes where a satellite **should have been** at the image timestamp.

Steps:
1. Load metadata JSON (timestamp)
2. Load TLE (Two-Line Element)
3. Propagate orbit using SGP4
4. Save orbit prediction report to outputs


In [ ]:
!pip -q install sgp4

import os, json, math
from datetime import datetime, timezone
from sgp4.api import Satrec, jday


In [ ]:
PROJECT_ROOT = "/content/orbitcheck"

META_DIR = os.path.join(PROJECT_ROOT, "data/raw/metadata")
TLE_DIR  = os.path.join(PROJECT_ROOT, "data/raw/tle")
REPORT_DIR = os.path.join(PROJECT_ROOT, "data/outputs/reports")

os.makedirs(TLE_DIR, exist_ok=True)
os.makedirs(REPORT_DIR, exist_ok=True)

print("META_DIR:", META_DIR)
print("TLE_DIR :", TLE_DIR)
print("REPORT_DIR:", REPORT_DIR)


In [ ]:
meta_files = [f for f in os.listdir(META_DIR) if f.endswith(".json")]
if len(meta_files) == 0:
    raise ValueError("No metadata JSON found. Run Notebook 01 first.")

print("Available metadata files:")
for f in meta_files:
    print(" -", f)

meta_filename = input("\nEnter metadata filename (example: sample1.json): ").strip()
meta_path = os.path.join(META_DIR, meta_filename)

with open(meta_path, "r") as f:
    meta = json.load(f)

timestamp = meta["timestamp_utc"]

print("\n Loaded Metadata:")
print(json.dumps(meta, indent=2))
print("\n Timestamp used:", timestamp)


In [ ]:
# Choose which satellite TLE you want to use
# "ISS" works for testing.
# Later can use Sentinel-2A or any Earth observation satellite.
SATELLITE_NAME = "ISS"

tle_path = os.path.join(TLE_DIR, f"{SATELLITE_NAME}.tle")

# Example ISS TLE (works for demo)
tle_line1 = "1 25544U 98067A   24055.51800926  .00016717  00000+0  10270-3 0  9995"
tle_line2 = "2 25544  51.6417  56.7071 0003430  80.7118  36.5230 15.49781609441430"

with open(tle_path, "w") as f:
    f.write(tle_line1 + "\n")
    f.write(tle_line2 + "\n")

print("Saved TLE:", tle_path)
print(tle_line1)
print(tle_line2)


In [ ]:
def propagate_tle(tle_line1, tle_line2, utc_time_str):
    """
    Propagate satellite position using SGP4.

    Returns:
    - position vector r (km)
    - velocity vector v (km/s)
    - error code
    """
    sat = Satrec.twoline2rv(tle_line1, tle_line2)

    dt = datetime.strptime(utc_time_str, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc)
    jd, fr = jday(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)

    e, r, v = sat.sgp4(jd, fr)
    return e, r, v


In [ ]:
EARTH_RADIUS_KM = 6371.0

def norm(vec):
    return math.sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)

e, r, v = propagate_tle(tle_line1, tle_line2, timestamp)

if e != 0:
    raise RuntimeError(f"SGP4 error code: {e}")

orbital_radius = norm(r)
altitude_km = orbital_radius - EARTH_RADIUS_KM

print("\nOrbit Propagation Output")
print("Position (TEME, km):", r)
print("Velocity (TEME, km/s):", v)
print(f"Approx altitude: {altitude_km:.2f} km")


In [ ]:
orbit_report = {
    "image_id": meta["image_id"],
    "timestamp_utc": timestamp,
    "satellite_name": SATELLITE_NAME,
    "tle": {
        "line1": tle_line1,
        "line2": tle_line2
    },
    "position_teme_km": {
        "x": r[0], "y": r[1], "z": r[2]
    },
    "velocity_teme_km_s": {
        "x": v[0], "y": v[1], "z": v[2]
    },
    "approx_altitude_km": altitude_km
}

out_path = os.path.join(REPORT_DIR, f"{meta['image_id']}_orbit_report.json")
with open(out_path, "w") as f:
    json.dump(orbit_report, f, indent=2)

print("\nOrbit report saved:", out_path)
print(json.dumps(orbit_report, indent=2))


In [ ]:
print("\nReports folder contains:")
print(os.listdir(REPORT_DIR))
